In [1]:
using StatsBase, Distributions
using Gurobi, Ipopt
using Test
using MLKernels
using LinearAlgebra, Random
using Suppressor
using JuMP

include("../src/SVDD.jl")
include("../test/test_utils.jl")

generate_mvn_with_outliers (generic function with 4 methods)

### Setup

Generate data

In [2]:
data, labels = generate_mvn_with_outliers(2, 100, 123, true, false);

Initialize SVDD

In [3]:
C_SVDD = 1.0

pools = fill(:U, size(data,2));
params = Dict(:C => C_SVDD)
init_strategy = SVDD.FixedParameterInitialization(MLKernels.GaussianKernel(0.1), C_SVDD)

#suppress deprecation warnings of MLKernels
@suppress begin      
    global svdd = SVDD.instantiate(SVDD.VanillaSVDD, data, pools, params)
    SVDD.initialize!(svdd, init_strategy)
end

K = svdd.K;

In [4]:
# some testing with JuMP 0.19-alpha because of regressions in v0.18.4
function SVDD.solve!(model::SVDD.VanillaSVDD, solver)
    QP = Model(with_optimizer(solver))
    @variable(QP, 0 <= α[1:size(K,1)] <= model.C);

    @objective(QP, Max, sum(α[i]*K[i,i] for i in eachindex(α)) - sum(α[i]*α[j] * K[i,j] for i in eachindex(α) for j in eachindex(α)));
    @constraint(QP, sum(α) == 1);
    @time JuMP.optimize!(QP)

    model.alpha_values = JuMP.result_value.(α)
    return JuMP.termination_status(QP)
end

In [5]:
SVDD.fit!(svdd, Gurobi.Optimizer)

  9.093894 seconds (16.42 M allocations: 824.625 MiB, 7.22% gc time)


Success::TerminationStatusCode = 0

#### SMO

In [6]:
function takeStep!(α, i1, i2, K, C_SVDD)
    i1 == i2 && return false
        
    L = max(0, α[i1] + α[i2] - C_SVDD)
    H = min(C_SVDD, α[i1] + α[i2])
    (abs(L - H) < OPT_PRECISION) && return false
    
    Δ = α[i1] + α[i2]
    C(i) = sum(α[j]*K[i,j] for j in eachindex(α) if !(j in [i1, i2]))

    alpha2 = (2*Δ*(K[i1, i1] - K[i1,i2]) + C(i1) - C(i2) - K[i1, i1] + K[i2, i2]) / (2*K[i1,i1] - 4*K[i1,i2] + 2*K[i2,i2])
                    
    if alpha2 > H
        alpha2 = H
    elseif alpha2 < L 
        alpha2 = L
    end

    if abs(α[i2] - alpha2) < OPT_PRECISION * (alpha2 + α[i2] + OPT_PRECISION) # p. 10 Platt SMO
        return false
    else
        alpha1 = Δ - alpha2
        α[i1] = alpha1
        α[i2] = alpha2              
        return true
    end
end

takeStep! (generic function with 1 method)

In [7]:
function update_model(α, K, C_SVDD)
    sv_larger_than_zero = α .> OPT_PRECISION 
    sv_smaller_than_C = α .< (C_SVDD - OPT_PRECISION)
        
    sv_larger_than_zero_idx = findall(sv_larger_than_zero)
    const_term = sum(α[i] * α[j] * K[i,j] for i in sv_larger_than_zero_idx for j in sv_larger_than_zero_idx)
    distances_to_center = [K[z, z] - 2 * sum(α[i] * K[i, z] for i in sv_larger_than_zero_idx) + const_term for z in eachindex(α)]
    
    ## see revisit to SVDD
    if any(sv_larger_than_zero .& sv_smaller_than_C)
        R = mean(distances_to_center[sv_larger_than_zero .& sv_smaller_than_C])
    else
        R = (minimum(distances_to_center[sv_larger_than_zero]) + maximum(distances_to_center[sv_larger_than_zero])) / 2
    end

    distances_to_decision_boundary = distances_to_center .- R
    return (distances_to_center, distances_to_decision_boundary, R)
end

update_model (generic function with 1 method)

In [8]:
function violates_KKT_condition(i2, distances_to_decision_boundary, α, C_SVDD)
    p1 = (α[i2] > OPT_PRECISION) && (distances_to_decision_boundary[i2] < -OPT_PRECISION) # inlier, but alpha > 0
    p2 = (α[i2] < C_SVDD - OPT_PRECISION) && (distances_to_decision_boundary[i2] > OPT_PRECISION) # outlier, but alpha != C
    return p1 || p2
end

violates_KKT_condition (generic function with 1 method)

In [9]:
function second_choice_heuristic(i2, α, distances_to_center, C_SVDD) # Eq. (4.8)
    SV_nb = (α .> OPT_PRECISION) .& (α .< C_SVDD - OPT_PRECISION)
    if !any(SV_nb)
        SV_nb = α .> OPT_PRECISION
    end
    findall(SV_nb)[findmax(abs.(distances_to_center[i2] .- distances_to_center[SV_nb]))[2]] 
end

second_choice_heuristic (generic function with 1 method)

In [10]:
function examineExample!(α, i2, distances_to_center, C_SVDD, K)
    
    # use the second choice heuristic
    i1 = second_choice_heuristic(i2, α, distances_to_center, C_SVDD)
    takeStep!(α, i1, i2, K, C_SVDD) && return true

    # loop over all non-zero and non-C alpha, starting at random position
    candidates = findall((α .> OPT_PRECISION) .& (α .< C_SVDD - OPT_PRECISION))
    if !isempty(candidates)
        for i1 in shuffle(candidates)
            takeStep!(α, i1, i2, K, C_SVDD) && return true
        end
    end
    
    # loop over all
    for i1 in shuffle(eachindex(α))
        takeStep!(α, i1, i2, K, C_SVDD) && return true
    end
    return false
end

examineExample! (generic function with 1 method)

In [11]:
function initialize_alpha(data, C_SVDD)
    n_init = trunc(Int, 1 / (C_SVDD)) + 1
    α = fill(0.0, size(data,2))
    α[sample(1:size(data,2), n_init, replace=false)] .= 1 / n_init
    @assert sum(α) ≈ 1
    @assert all(α .<= C_SVDD)
    return α
end

initialize_alpha (generic function with 1 method)

In [12]:
function smo(;max_iterations = 10000)
    
    α = initialize_alpha(data, C_SVDD)
    distances_to_center, distances_to_decision_boundary, R = update_model(α, K, C_SVDD)
    
    iter = 0
    
    function build_result(status, msg)
        println("Exit with status $status. (" * msg * ")")
       (α, distances_to_decision_boundary, distances_to_center, R)
    end
    
    while iter < max_iterations
        black_list = Set{Int}()
        iter += 1
        
        # scan over all data
        KKT_violation_all_idx = filter(i -> violates_KKT_condition(i, distances_to_decision_boundary, α, C_SVDD) && i ∉ black_list, eachindex(α))
        
        isempty(KKT_violation_all_idx) && return build_result(:Optimal, "No more KKT_violations.")

        i2 = sample(KKT_violation_all_idx)
        if examineExample!(α, i2, distances_to_center, C_SVDD, K)
            distances_to_center, distances_to_decision_boundary, R = update_model(α, K, C_SVDD)
        else
            push!(black_list, i2)
        end

        # scan over SV_nb
        SV_nb = (α .> OPT_PRECISION) .& (α .< C_SVDD - OPT_PRECISION)
        KKT_violations_in_SV_nb = filter(i -> violates_KKT_condition(i, distances_to_decision_boundary, α, C_SVDD) && i ∉ black_list, findall(SV_nb))
        
        while length(KKT_violations_in_SV_nb) > 0 && iter < max_iterations
            iter += 1

            i2 = sample(KKT_violations_in_SV_nb)
            if examineExample!(α, i2, distances_to_center, C_SVDD, K)
                distances_to_center, distances_to_decision_boundary, R = update_model(α, K, C_SVDD)
            else
                push!(black_list, i2)
            end
            KKT_violations_in_SV_nb = filter(i -> violates_KKT_condition(i, distances_to_decision_boundary, α, C_SVDD) && i ∉ black_list, findall(SV_nb))
        end
    end
    
    return build_result(:NotOptimal, "Reached max number of iterations.") 
end

smo (generic function with 1 method)

In [13]:
OPT_PRECISION = 1e-2

0.01

In [14]:
Random.seed!(4)
@time α, distances_to_decision_boundary, distances_to_center, R = smo(max_iterations = 1000);

Exit with status Optimal. (No more KKT_violations.)
  3.211198 seconds (5.76 M allocations: 290.339 MiB, 6.35% gc time)


In [15]:
sv_larger_than_zero_idx = findall(α .> OPT_PRECISION)

primal_obj = R + sum(distances_to_decision_boundary[distances_to_decision_boundary .> OPT_PRECISION] * C_SVDD)
dual_obj = sum(α[i] * K[i,i] for i in sv_larger_than_zero_idx) - sum(α[i] * α[j] * K[i,j] for i in sv_larger_than_zero_idx for j in sv_larger_than_zero_idx)
duality_gap = primal_obj - dual_obj
            
@show primal_obj, dual_obj, duality_gap;

(primal_obj, dual_obj, duality_gap) = (0.016241665164839974, 0.013881659999715135, 0.0023600051651248392)
